In [ ]:
import numpy as np
import astropy.io.fits as pf
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.gaussian_process import GaussianProcessRegressor
import sklearn.gaussian_process.kernels as kernels
import forAustin as fa
%matplotlib inline

In [ ]:
fits = pf.open('/data4/paper/fox/DES/austinFull.fits')
fits.info()
nExposure = 432
exposure = fa.getExposure(fits, nExposure, polyOrder=3)

In [ ]:
ind = np.where(exposure['hasGaia'])[0]
u = np.take(exposure['u'], ind)
v = np.take(exposure['v'], ind)
dx = np.take(exposure['dx'], ind)
dy = np.take(exposure['dy'], ind)
E = np.take(exposure['measErr'], ind)

X = np.vstack((u, v)).T
Y = np.vstack((dx, dy)).T

Xtrain, Xtest, Ytrain, Ytest, Etrain, Etest= train_test_split(X, Y, E, test_size=0.10, random_state=1)

In [ ]:
kernel = kernels.RBF(np.array([5, 5])) + kernels.WhiteKernel(Etrain.mean())

In [ ]:
GP = GaussianProcessRegressor(kernel=kernel, n_restarts_optimizer=10)
GP.fit(Xtrain, Ytrain)

In [ ]:
Ypred, cov = GP.predict(Xtest, return_cov=True)

In [ ]:
variance = np.diag(cov)
sigma = np.sqrt(variance)

In [ ]:
plt.figure(figsize=(14, 7))
plt.title("Testing Residuals vs Predicted Residuals")
# plt.xlim((-300, 300))
# plt.ylim((-100, 100))

plt.scatter(Ypred[1, 0], Ypred[1, 1], label="Predicted Set")
plt.scatter(Ytest[1, 0], Ytest[1, 1], label="Test Set")
plt.legend()
plt.show()

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Vector Field of Predicted Set")
plt.quiver(Xtest[:, 0], Xtest[:, 1], Ypred[:, 0], Ypred[:, 1])

In [ ]:
plt.figure(figsize=(8, 8))
plt.title("Vector Field of Test Set")
plt.quiver(Xtest[:, 0], Xtest[:, 1], Ytest[:, 0], Ytest[:, 1])